<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML241ENSkillsNetwork31576874-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Model-agnostic Explanations**


Estimated time needed: **45** minutes


In this lab, we will first train a random forest model to predict if employees are looking for a job change, then we want to interpret the trained model in order to understand how exactly it makes predictions. Since random forest model is normally very complex to understand, we will just treat it as a black-box model first. Then, you will have the practice opportunities to apply various model-agnostic explanation methods to explain the black-box model.


## Objectives


After completing this lab you will be able to:


*   Calculate Permutation Feature Importance
*   Use Partial Dependency Plot to illustrate relationships between feature and outcomes
*   Build Global Surrogate Models
*   Build Local Surrogate Models using `LIME`


***


## Setup


Let's first import required Python packages for this lab:


In [ ]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
# !mamba install -qy pandas==1.3.3 numpy==1.21.2 ipywidgets==7.4.2 scipy==7.4.2 tqdm==4.62.3 matplotlib==3.5.0 seaborn==0.9.0

# install imbalanced-learn package
!pip install lime==0.2.0.1

# Note: If your environment doesn't support "!mamba install", use "!pip install" 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 28.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283839 sha256=047995627bed4809c58d5586e1647a54e8863392fa044703d870000bee98684f
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime


And then import the required Python packages.


In [ ]:
## Import packages here
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import lime.lime_tabular

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, export_text, export_graphviz, plot_tree
from sklearn.inspection import permutation_importance, plot_partial_dependence

ImportError: cannot import name '_print_elapsed_time' from 'sklearn.utils' (/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/__init__.py)

Then, let's load the dataset to be used in this lab.


In [ ]:
url="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML201EN-SkillsNetwork/labs/module_4/datasets/hr_new_job_processed.csv"
job_df=pd.read_csv(url)

In [ ]:
job_df.describe()

,city_development_index,training_hours,company_size,experience,education_level,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,target
count,11504.000000,11504.000000,11504.000000,11504.000000,11504.000000,11504.000000,11504.000000,11504.000000,11504.000000,11504.000000,11504.000000,11504.000000
mean,0.806056,64.380421,2.748997,9.530985,2.131568,0.029180,0.042537,0.023619,0.005557,0.045442,0.853665,0.500000
std,0.128302,59.048600,1.783676,6.733267,0.627610,0.161802,0.197495,0.147348,0.070875,0.200717,0.342167,0.500022
min,0.448000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.682000,23.000000,2.000000,4.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,0.878000,46.000000,2.000000,8.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.500000
75%,0.920000,88.000000,3.000000,15.000000,2.247823,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,0.949000,336.000000,7.000000,21.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


The dataset contains the following features (predictors):

*   `city_ development index` : Developement index of the city, ranged from 0 to 1
*   `training_hours`: Training hours completed, ranged from 0 to 336
*   `company_size`: Size of the current company, ranged from 0 to 7 where 0 means less than 10 employees and 7 means more than 10,000 employees
*   `education_level`: Education level of the candidate, ranged from 0 to 4 where 0 means Primary School and 4 means Phd
*   `experience`: Total experience in years, ranged from 0 to 21
*   `company_type` : *Categorical column* with one-hot encodings. Type of current company:  'Pvt Ltd', 'Funded Startup', 'Early Stage Startup', 'Other', 'Public Sector', 'NGO'

and the prediction outcome is:

*   `target`: `0` – Not looking for a job change, `1` – Looking for a job change


The predictive task is a straightforward binary classification task, more specifically, we want to use an employee's profile features to predict if he/she is looking for a job change or not.


## Build a Random Forest classifier as the Black-box model


### Split the training and testing datasets


In [ ]:
X = job_df.loc[:, job_df.columns != 'target']
y = job_df[['target']]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state = 12)

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1609: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1609: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review you

Now let's train a `Random Forest` model with the following preset arguments. If you like, you may also use hyperparameter tuning methods to tune these parameters yourself.

*   `random_state = 0` as a random seed to reproduce the result
*   `max_depth = 25` means the max depth of a tree should be less than 25
*   `max_features = 10` means the random forest includes max 10 features
*   `n_estimators = 100` means total 100 trees will be built
*   `bootstrap = True` means bootstrap samples will be used to build trees


In [ ]:
# Define a black-box random forest model
black_box_model = RandomForestClassifier(random_state = 123, max_depth=25, 
                             max_features=10, n_estimators=100, 
                             bootstrap=True)
# Train the model
black_box_model.fit(X_train, y_train.values.ravel())

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/ensemble/forest.py:487: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/tree/tree.py:149: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features=10, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

Next, let's make some predictions and evalute the model using `AUC` score:


In [ ]:
#predicting test set
y_blackbox = black_box_model.predict(X_test)

In [ ]:
metrics.roc_auc_score(y_test, y_blackbox)

0.8131250708268802

Your AUC score should be around `0.81`, which indicates the model is doing a very good job in the test dataset.


Now we have a black-box random forest model trained, we want to use various model-agnostic methods to explain it.


Note that if you prefer other binary classification models such as XGBoosting, you could train one here by yourself as well, and it won't affect
the subsequent steps since our explanations are all model-agnostic.


## Permutation Feature Importance


One common way to explain a machine learning model is via finding its important features and **permutation feature importance** is a popular method to calculate feature importance.


The basic idea of permutation feature importance is we shuffle interested feature values and make predictions using the shuffled values.
The feature importance will be measured by calculating the difference between the prediction errors before and after permutation.


In this lab, we will use `permutation_importance` function provided by `sklearn` to easily calculate importance for all features.


You can call `permutation_importance` with the following key arguments:

*   `estimator` the model to be estimated
*   `X` training data X
*   `y` target labels y
*   `n_repeats`, Number of times to permute a feature, each permutation generates an importance value


In [ ]:
# Use permutation_importance to calculate permutation feature importances
feature_importances = permutation_importance(estimator=black_box_model, X = X_train, y = y_train, n_repeats=5,
                                random_state=123, n_jobs=2)

ImportError: cannot import name '_print_elapsed_time' from 'sklearn.utils' (/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/__init__.py)

Let's take a look at the generated importance results:


In [ ]:
feature_importances.importances.shape

In [ ]:
feature_importances.importances

It is a `11 x 5` numpy array, 11 means we have 11 features, and 5 represents the total number of permutation times.

For each permutation, we will have a list of importance score calculated for each feature. The value represents the portion of increased prediction errors, important features will have larger values.


However, the feature importance array above is very difficult to comprehend, let's sort and visualize it:


In [ ]:
def visualize_feature_importance(importance_array):
    # Sort the array based on mean value
    sorted_idx = importance_array.importances_mean.argsort()
    # Visualize the feature importances using boxplot
    fig, ax = plt.subplots()
    fig.set_figwidth(16)
    fig.set_figheight(10)
    fig.tight_layout()
    ax.boxplot(importance_array.importances[sorted_idx].T,
               vert=False, labels=X_train.columns[sorted_idx])
    ax.set_title("Permutation Importances (train set)")
    plt.show()

In [ ]:
visualize_feature_importance(feature_importances)

NameError: name 'feature_importances' is not defined

Now you should see a box plot show ranked feature importances, and we can see the most important features are `city_development_index`, `company_size`, `training_hours`, `experiences`, `education_level`, and so on, and you should have a general understanding of how the black-box model determines if an employee is looking for a new job or not.


### Exercise: Use a different `n_repeats=10` to calculate and visualize feature importance values


In [ ]:
# Type your answer here
# Update n_repeats=10 and recalculate and visualize feature importance


<details><summary>Click here for a sample solution</summary>

```python
feature_importances = permutation_importance(estimator=black_box_model, X = X_train, y = y_train, n_repeats=10,
                                random_state=123, n_jobs=2)

visualize_feature_importance(feature_importances)

```

</details>


## Partial Dependency Plot (PDP)


Partial Dependency Plot (PDP) is an effective way to illustrate the relationship between an interested feature and the model outcome. It essentially visualizes the marginal effects of a feature, that is, shows how the model outcome changes when a specific feature changes in its distribution.


Since a machine learning model may include many features, and it is not feasible to create PDP for every single feature. Thus, we normally first find the most important features via ranking their feature importances. Then, we can only focus PDP on those important features.


From the previous step, we know some important features are `city_development_index`, `company_size`, `experience`, `education_level`, and we can easily create PDP for those features using `plot_partial_dependence` in `sklearn` package.


Let's first try to create PDP for features `city_development_index`, `experience`:


In [ ]:
# Important features
important_features = ['city_development_index', 'experience']
"arguments: "
" - estimator: the black box model"
" - X is the training data X"
" - features are the important features we are interested"
plot_partial_dependence(estimator=black_box_model, 
                        X=X_train, 
                        features=important_features,
                        random_state=123)

NameError: name 'plot_partial_dependence' is not defined

Then you should see two PDPs are plotted for `city_development_index` and `experience`. They all have roughly negative linear relationship betweens the outcome, for example, if an employee is in a well-developed city and has a lot of experiences, then he/she is unlikely to change the current job.


### Exercise: Create PDPs for other important features such as `company_size`, `education_level`, `training_hours`, and others


In [ ]:
# Type your solution here
# Create PDPs for other important features


<details><summary>Click here for a sample solution</summary>

```python
important_features = ['company_size', 'education_level', 'training_hours']
    
plot_partial_dependence(estimator=black_box_model, 
                        X=X_train, 
                        features=important_features,
                        random_state=123)
```

</details>


## Global Surrogate Model


Now you have explored how to explain the black-box model via analyzing its features. Next, we will learn how to explain it via approximate of their inputs and outputs with a global surrogate model.


We will be training two self-interpretable models: 1) a logistic regression model and 2) a decision tree models using the inputs and outputs from the black-box model


You can follow these general steps to build a global surrogate model:

*   First, we select a dataset `X_test` as input

*   Then, we use the black-box model to make predictions `y_blackbox` using the `X_test`

*   With both training data and labels ready, we can use them to train a simple logistic regression model and a decision tree model

*   The surrogate model outputs its own predictions `y_surrogate`

*   Lastly, we can measure the difference between `y_surrogate` and `y_blackbox` using an accuracy score to determine how well the surrogate model approximating the black-box model


![global_surrogate](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML241EN-SkillsNetwork/labs/module6\_model_interpretability/images/global_surrogate.png)


### Logistic regression surrogate model


In order to compare the coefficients of the logistic regression model directly, we want to normalize the input X first.


In [ ]:
# normalize X_test
min_max_scaler = StandardScaler()
X_test_minmax = min_max_scaler.fit_transform(X_test)

Then, we can train a logistic regression model with an `L1` regularizer to simplify the model and increase interpretability. Note that `y_blackbox` is coming from the predictions of black-box model.


In [ ]:
lm_surrogate = LogisticRegression(max_iter=1000, 
                                  random_state=123, penalty='l1', solver='liblinear')
lm_surrogate.fit(X_test_minmax, y_blackbox)

With the surrogate model trained, we can generate predictions using `X_test`,


In [ ]:
y_surrogate = lm_surrogate.predict(X_test_minmax)

and calculate how accurate the surrogate model approximates the black-box model.


In [ ]:
metrics.accuracy_score(y_blackbox, y_surrogate)

The score is around 0.75 which means the logistic regression surrogate model was able to reproduce about 75% of the original black-box model correctly.


Next, we can start interpreting the much simpler logistic regression model `lm_surrogate` via analyzing its feature coefficients. We defined a function called `get_feature_coeffs` to extract and sort feature coefficients from `lm_surrogate` model:


In [ ]:
# Extract and sort feature coefficients
def get_feature_coefs(regression_model):
    coef_dict = {}
    # Filter coefficients less than 0.01
    for coef, feat in zip(regression_model.coef_[0, :], X_test.columns):
        if abs(coef) >= 0.01:
            coef_dict[feat] = coef
    # Sort coefficients
    coef_dict = {k: v for k, v in sorted(coef_dict.items(), key=lambda item: item[1])}
    return coef_dict

In [ ]:
coef_dict = get_feature_coefs(lm_surrogate)
coef_dict

We can get a coefficient dict object whose keys are features and values are coefficients, but such dict object is not easy to understand so let's just visualize it using a bar chart:


In [ ]:
# Generate bar colors based on if value is negative or positive
def get_bar_colors(values):
    color_vals = []
    for val in values:
        if val <= 0:
            color_vals.append('r')
        else:
            color_vals.append('g')
    return color_vals

# Visualize coefficients
def visualize_coefs(coef_dict):
    features = list(coef_dict.keys())
    values = list(coef_dict.values())
    y_pos = np.arange(len(features))
    color_vals = get_bar_colors(values)
    plt.rcdefaults()
    fig, ax = plt.subplots()
    ax.barh(y_pos, values, align='center', color=color_vals)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(features)
    # labels read top-to-bottom
    ax.invert_yaxis()  
    ax.set_xlabel('Feature Coefficients')
    ax.set_title('')
    plt.show()
    

Let's call `visualize_coefs` function to visualize the coefficients dict:


In [ ]:
visualize_coefs(coef_dict)

From the bar chart above, you can immediately find those important features with negative effects such as `city_development_index` and `experience`, and those have positive effects such as education_level or if the company is a `Pvt Ltd`.


### Exercise: Build a global surrogate model using decision tree


In [ ]:
# Type your answer here
# Define a decision tree model
tree_surrogate = DecisionTreeClassifier(random_state=123, 
                                         max_depth=5, 
                                         max_features=10)

<details><summary>Click here for a sample solution</summary>

```python
tree_surrogate = DecisionTreeClassifier(random_state=123, 
                                         max_depth=5, 
                                         max_features=10)
```

</details>


In [ ]:
# Type your answer here
# Train the decision tree model with X_test and y_blackbox, and make predictions on X_test
tree_surrogate.fit(X_test, y_blackbox)
y_surrogate = tree_surrogate.predict(X_test)

<details><summary>Click here for a sample solution</summary>

```python
tree_surrogate.fit(X_test, y_blackbox)
y_surrogate = tree_surrogate.predict(X_test)
```

</details>


In [ ]:
# Type your answer here
# Measure the difference between
metrics.accuracy_score(y_blackbox, y_surrogate)

<details><summary>Click here for a sample solution</summary>

```python
metrics.accuracy_score(y_blackbox, y_surrogate)
```

</details>


Now, you have trained the tree surrogate model, you could interprete it by export and print the tree:


In [ ]:
tree_exp = export_text(tree_surrogate, feature_names=list(X_train.columns))

In [ ]:
print(tree_exp)

## Local interpretable model-agnostic explanations (LIME)


Global surrogate models may have large prediction inconsistency between the complex black-box model and the simple surrogate models or there are many instance groups or clusters in the dataset which make the surrogate model more generalized to those different patterns and lose the interpretability on a specific data group.

On the other hand, we are also interested in how black-box models make predictions on some representative instances. By understanding these very typical examples, we can sometimes obtain valuable insights without understanding the model’s behaviors on the entire dataset.


Next, you will be building a local surrogate model using LIME method whose general steps are shown in the following flowchart:


![Local interpretable model-agnostic method](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-ML241EN-SkillsNetwork/labs/module6\_model_interpretability/images/lime.png)


We can use a open source [lime](https://github.com/marcotcr/lime) package to easily build a LIME explainer the our black-box model, let's get started.


First, we need to define a `LimeTabularExplainer` to explain those predictive models built on structured/tabular datasets, like the job-changing prediction dataset we are using.


Note although LIME algorithm is a local surrogate model, it still also requires a training dataset containing your interested data instances. So that it can perform uniform sampling (feature permutations) around the interested data instances to generate the artificial dataset for the actual surrogate model training process.


In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(
    # Set the training dataset to be X_test.values (2-D Numpy array)
    training_data=X_test.values,
    # Set the mode to be classification
    mode='classification',
    # Set class names to be `Not Changing` and `Changing`
    class_names = ['Not Changing', 'Changing'],
    # Set feature names
    feature_names=list(X_train.columns),
    random_state=123,
    verbose=True)

Now, let's try to select an interested employee from `X_test`, and we want to understand its prediction using the `LimeTabularExplainer`.


In [ ]:
instance_index = 19
selected_instance = X_test.iloc[[instance_index]]
lime_test_instance = selected_instance.values.reshape(-1)
selected_instance

Let's make a quick summary about this employee:

*   His/her city is well-developed with a city development index > 0.9
*   His/her training hour is 74 hours
*   His/her company is a very big company, 7 means more than 10,000 employees
*   His/her experience is more than 16 years
*   His/her company is a Pvt Ltd (Private) company
*   His/her has Master's degree(s)

and our black-box model predicts its probability of changing a job is `0.03`, that is, very unlikely to leave his or her current job.


Then, let's use `LimeTabularExplainer` to explain why the black-box model thinks this employee won't leave his/her current job.


In [ ]:
exp = explainer.explain_instance(
                                 # Instance to explain
                                 lime_test_instance, 
                                 # The prediction from black-box model
                                 black_box_model.predict_proba,
                                 # Use max 10 features
                                 num_features=10)
exp.as_pyplot_figure();

`LimeTabularExplainer` outputs a bar chart similar to the coefficient or feature importance chart we plotted before.
From its output, we can easily interpret why the black-box thinks this employe won't change job, based on the following main factors:

*   His/her company is a very big company
*   His/her city is well-developed with city development
*   His/her highest degree is Master or above
*   His/her experience is more than 15 years
*   His/her company is not NGO or Startup

This interpretation is also aligned with our common sense, that is, if a well-educated employee has been working in a very big/good private company, located in a big city, for more than 15 years, then he/she is probably very satisfied with current job and does not want to change it.


### Exerice: Find other data instances and use LimeTabularExplainer to explain their predictions of black-box model


In [ ]:
# Update instance_index, and rerun the explainer.explain_instance() method


## Next Steps


By now you have learned and applied various model-agnostic explanation algorithms such as Permutation Feature Importance, PDP, Global Surrogate Model, LIME, and others in this lab. There are many other such methods such as Feature Interactions, Individual Conditional Expectation, SHAP values, and so on, and we do not have enough time to explain them all in this course.

We list the references to other popular model explanation methods which you may be interested:

*   [Predictive learning via rule ensembles](https://arxiv.org/abs/0811.1679?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML241ENSkillsNetwork31576874-2022-01-01)
*   [A Unified Approach to Interpreting Model Predictions](https://arxiv.org/abs/1705.07874?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML241ENSkillsNetwork31576874-2022-01-01)
*   [Peeking Inside the Black Box: Visualizing Statistical Learning with Plots of Individual Conditional Expectation](https://arxiv.org/abs/1309.6392?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML241ENSkillsNetwork31576874-2022-01-01)


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML241ENSkillsNetwork31576874-2022-01-01)


### Other Contributors


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |          |     |            |         |
| ----------------- | ------- | ---------- | --------------------------- | -------- | --- | ---------- | ------- |
| 2021-8-23         | 1.0     | Yan        | Created the initial version | 2022-2-8 | 1.1 | Steve Hord | QA pass |


Copyright © 2021 IBM Corporation. All rights reserved.
